<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/University_of_Los_Andes_logo.svg/1200px-University_of_Los_Andes_logo.svg.png" width="70" align="left"> 
  <h1> DEPARTAMENTO DE INGENIERIA INDUSTRIAL <br>
  Estrucuturación de Redes Sociales 202120 <br> </h1>
 
 **PROFESORES:** Felipe Montes - Andrés Felipe Usceche - Gianfranco Vietto - Ana María Guerra - Martha Blanco - Ana María Jaramillo

<center> <bold> Integrantes </bold> </center>

|Apellidos|Nombres|Código|Login|
|:---:|:---:|:---:|:---:|
|*Ávila Vera*|*Daniel*|*201530842*|*da.avila*|
|*Apellidos 2*|*Nombres 2*|*Código 2*|*login 2*|

<h1><center> Taller 2 </center></h1>
<h2><center>Diferenciar métodos para recolectar información de redes</center></h2>

Esta actividad va a estar basada en el estudio realizado por Mendoza-Palechor y de la Hoz-Manotas (2019). Esta base de datos recopila información de 2111 niñ@s de Colombia, Perú y México. Para estos niñ@s se realizó una estimación de los niveles de obesidad basado en hábitos de comida y sus condiciones de actividad física. 
En este ejercicio vamos a comparar la diferencia entre dos metodologías de muestreo. La primera es el muestreo aleatorio y la segunda es el muestreo por bola de nieve en redes. El objetivo de esta actividad es entender la importancia del muestreo en redes para capturar las relaciones y la estructura de la red a diferencia de las metodologías de muestreo estadístico que lo que buscan es representatividad muestral.

Suponga que un grupo de la OMS le ha encargado analizar desde la perspectiva de las redes sociales la muestra del estudio de Mendoza-Palechor y de la Hoz-Manotas. Quieren que usted realice un estudio que permita determinar cuáles son los niños más influenciadores que puedan ser utilizados para compartir un programa de actividad física con sus compañeros. El problema con el equipo es que no tiene suficiente presupuesto para preguntarle a cada uno de los niños cuáles son sus amigos. Por lo tanto, le han pedido que cree una muestra a partir de la población de niños y con base en esa muestra realice el análisis de redes para determinar a los niños más influenciadores. Para esto, usted se ha propuesto seguir los siguientes pasos para cumplir el requerimiento.

In [213]:
## Librerías a usar
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
from scipy.stats import norm

In [214]:
## Cargar la red
nodos=pd.read_csv("./nodos_base.csv",index_col="id")
arcos=pd.read_csv("./arcos_base.csv")

1.	Dada la restricción presupuestal, usted sabe que lo primero que hay que hacer es determinar el tamaño de la muestra. Un aspecto relevante que le han solicitado es que la muestra que usted seleccione mantenga las proporciones de sexo de la población total. Calcule el tamaño de la muestra utilizando como referencia un nivel de confianza del 5% y un margen de error del 5%.

In [215]:
# Proporción de mujeres
perc_m = nodos[nodos.Gender == 'Female'].shape[0]/nodos.shape[0]
print(perc_m)

0.49407863571766936


In [216]:
#Tamaño de la muestra
p = perc_m
q = 1-p
e = 0.05
alpha = 0.05
Z = norm.ppf(alpha)
n = ((Z**2)*p*q)/(e**2)
sample_size = math.floor(n)
print(sample_size)

270


2.	El grupo de la OMS no está seguro sobre la relevancia de utilizar metodologías de muestreo en redes y cree que la mejor estrategia es realizar un muestro estratificado de acuerdo al sexo. Por lo cual le han solicitado que diseñe una muestra que cumpla las especificaciones del tamaño muestral. ¿Cuál es la edad promedio de los niños y las niñas en la muestra?

In [217]:
proportions = {'Female':perc_m, 'Male':1-perc_m}
dfs = []
for sex in proportions:
    sub_sample_size = round(sample_size*proportions[sex])
    sample_temp = nodos[nodos.Gender == sex].sample(n=sub_sample_size, random_state=2)
    dfs.append(sample_temp)
    
stratified_sample = pd.concat(dfs)

In [218]:
#promedio edad muestra mujeres
m_age_mean = stratified_sample[stratified_sample.Gender == 'Female'].Age.mean()
print(m_age_mean)

23.4205849849624


In [219]:
#promedio edad muestra hombres
h_age_mean = stratified_sample[stratified_sample.Gender == 'Male'].Age.mean()
print(h_age_mean)

24.862211766423354


3.	Dado que usted ya seleccionó su muestra, ahora va a reconstruir la red de estos niños. Para esto, usted le ha preguntado a cada uno de los niños en la muestra que le mencione al a lo sumo 20 de sus amigos con los que pasa la mayoría del tiempo. Esa información ya fue recopilada y se encuentra en el archivo de arcos (por simplicidad usted tiene la información de los arcos de todos los niños). De acuerdo al grado total, ¿quiénes son los 5 niños con mayor número de conexiones?

In [220]:
## Construir Red
G = nx.DiGraph()
G.add_edges_from(list(arcos.to_records(index=False)))

In [221]:
# Top 5 degree
print(sorted(G.degree, key=lambda x: x[1], reverse=True)[:5])

[(358, 42), (704, 38), (58, 37), (1711, 37), (1486, 36)]


4.	Usted recuerda que en el curso de Estructuración de Redes Sociales le enseñaron que el muestro estadístico tiene problemas a la hora de capturar las redes de los niños. Por lo tanto, usted le ha propuesto realizar de manera paralela al grupo de la OMS un muestreo por bola de nieve. El equipo aceptó su propuesta, sin embargo, le solicitaron que el número de niñ@s que capturara el muestro no fuera superior al tamaño de la muestra determinado previamente. ¿Cuál es la edad promedio de los niños y las niñas en la nueva muestra?

In [222]:
#Bola de Nieve
nodos_sample = nodos.sample(2, random_state=1)
arcos_sample = arcos[arcos.Source.isin(nodos_sample.index)]

while True: 
    nuevos_nodos = nodos[nodos.index.isin(arcos_sample.Target.unique())]
    nodos_sample_temp= nodos_sample.copy().append(nuevos_nodos).drop_duplicates()
    if nodos_sample_temp.shape[0] >270:
        break
    else:
        nodos_sample = nodos_sample_temp
        arcos_sample = arcos[arcos.Source.isin(nodos_sample.index)]

print(nodos_sample.shape[0])        

256


In [223]:
#Edad promedio niñas
m_age_mean = nodos_sample[nodos_sample.Gender == 'Female'].Age.mean()
print(m_age_mean)

24.521023347457625


In [224]:
#Edad promedio niños
h_age_mean = nodos_sample[nodos_sample.Gender == 'Male'].Age.mean()
print(h_age_mean)

24.28387237681159


5.	Usted va a replicar el ejercicio de reconstruir la red de los niños con esta nueva muestra. De acuerdo con el grado total, ¿quiénes son los 5 niños con mayor número de conexiones?

In [195]:
## Construir Red
G = nx.DiGraph()
G.add_edges_from(list(arcos_sample.to_records(index=False)))

In [196]:
# Top 5 degree
print(sorted(G.degree, key=lambda x: x[1], reverse=True)[:5])

[(1490, 25), (718, 24), (1527, 23), (1704, 23), (1743, 23)]


6.	El grupo de la OMS está complacido con los resultados, sin embargo, le han hecho la siguiente pregunta: ¿cuáles de los niños más influenciadores debería elegir: los del muestreo estadístico o los del muestreo en redes? Justifique su respuesta argumentado con base en algunas de las métricas de cada una de las redes.

In [225]:
# Escriba aquí su respuesta



Deberían elegir a los niños más influenciadores del muestro por bola de nieve. Esto se debe a que los vínculos entre los niños no siguen una distribución normal, por lo que asumir que todos tienen el mismo peso –como sucede en el muestreo estadístico– puede llegar a oscurecer a los actores centrales de esta red de relaciones. Por el contrario, el muestreo por bola de nieve facilita la captura de aquellos nodos que resaltan por su cantidad de relaciones en la red. 


7.	Exporte la red generada por el método de bola de nieve para realizar los siguintes pasos en Gephi:

In [133]:
## Exporten aquí
nodos_sample.to_csv('nodos_snowball.csv')
arcos_sample.to_csv('arcos_snowball.csv')

1.	Deberán cargar datos de las aristas en un nuevo proyecto en Gephi. No olviden que deben identificar si la red es dirigida o no y justificar porqué.

2.	Visualicen la red bajo un algoritmo, el que ustedes prefieran, y muestren como es esta primera visualización. (Deben exportarlo mediante la herramienta de previsualización)

Para nuestro estudio, queremos detectar si existen grupos entre los niños. Para esto vamos a usar el concepto de detección de comunidades. Una de las maneras de detectar comunidades es mediante la maximización del coeficiente de modularidad utilizando el algoritmo de Louvain. Dicho algoritmo consiste en asignar los nodos a comunidades de tal forma que se maximice la función de modularidad. La función de modularidad evalúa la proporción de arcos entre nodos que se encuentran en la misma comunidad con respecto a haber cconectado los nodos de manera aleatoria. 
•	Para más información sobre el algoritmo de Louvain referirse al artículo disponible en: https://arxiv.org/pdf/0803.0476.pdf o al artículo de Wikipedia https://es.wikipedia.org/wiki/M%C3%A9todo_de_Louvain . 
•	Para mayor información acerca de la modularidad favor referirse al capítulo 7.7.1. del libro de Newman.

Con lo anterior en mente, realicen el siguiente estudio en la red.

3.	Calculen la modularidad de toda la red y muestren el reporte de modularidad. <br>
    a.	¿Cuánto es la modularidad de la red? ¿Cómo interpretarían ese valor?<br>
    b.	¿Cuántas comunidades se identifican en la red?<br>

4.	Visualicen y estudien los grupos que detectaron anteriormente. Para esto ustedes deben seguir los siguientes pasos:<br>
    a.	Coloreen los nodos a partir de las comunidades detectadas.<br>
    b.	Cambien el tamaño de los nodos a partir del grado de estos.<br>
    c.	Apliquen la distribución “Force Atlas 2” y expandan el grafo para mejorar la visualización.<br>
    d.	Muestre el grafo resultante.<br>

5.	Una vez identificadas las comunidades en la red, identifiquen cuál de las comunidades es más fuerte y cuál más débil. Para esto ustedes deben:<br>
    a.	Encontrar cuál es la comunidad que tiene una mayor amistades.<br>
    b.	Identificar cuál es el grupo con mayor densidad de relaciones de pases entre los jugadores.<br>


#### Realice esta actividad enteramente en Python, y una vez terminen el estudio deberán presentar sus resultados en un informe autocontenido mostrado su desarrollo y análisis. Concluida la actividad, compriman en un zip su documento, el Notebook y el archivo de Gephi de soporte.